In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

from sklearn.decomposition import PCA

import numpy as np # linear algebra
import pandas as pd 

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
from sklearn.preprocessing import QuantileTransformer

In [3]:

data_dir = '../data/01_raw'
os.listdir(data_dir)

['train_targets_scored.csv',
 'sample_submission.csv',
 '.gitkeep',
 'train_features.csv',
 'test_features.csv',
 'train_targets_nonscored.csv']

In [4]:
train_features = pd.read_csv(data_dir+'/train_features.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir+'/train_targets_nonscored.csv')

test_features = pd.read_csv(data_dir+'/test_features.csv')
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')

In [5]:
seed = 42


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed)

In [6]:
# #Indiquer si valeur dans le range max, min

# import seaborn as sns
# data = pd.concat([train_features,test_features],axis=0)

# sns.distplot(data[data["cp_type"] == "ctl_vehicle"]["c-4"],label="normal")

# sns.distplot(data[data["cp_type"] == "trt_cp"]["c-4"],label="treated")
# plt.legend()
# plt.show()

In [7]:
# sum_targets = train_targets_scored[[c for c in train_targets_scored.columns if (c != "sig_id")]].sum().reset_index()

In [8]:
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_inhibitor",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_activator",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_agonist",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_antagonist",""))

In [9]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [10]:
# from scipy import stats

# train_features[GENES].apply(lambda x : stats.moment(x,moment=5),axis=1)

In [11]:
#RankGauss

# for col in (GENES + CELLS):

#     transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
#     vec_len = len(train_features[col].values)
#     vec_len_test = len(test_features[col].values)
#     raw_vec = train_features[col].values.reshape(vec_len, 1)
#     transformer.fit(raw_vec)

#     train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
#     test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [12]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [13]:
# # GENES
# n_comp = 600  #<--Update

# data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
# data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
# train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

# train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
# test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# # drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
# train_features = pd.concat((train_features, train2), axis=1)
# test_features = pd.concat((test_features, test2), axis=1)

In [14]:
# #CELLS
# n_comp = 50  #<--Update

# data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
# data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
# train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

# train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
# test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# # drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
# train_features = pd.concat((train_features, train2), axis=1)
# test_features = pd.concat((test_features, test2), axis=1)

In [15]:
train_features.shape

(23814, 876)

In [16]:
from scipy import stats

def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1) ## <==
        # df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1) ## <==
        # df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        #df['g_skew'] = df[features_g].skew(axis = 1)
        # df['g_q25'] = df[features_g].quantile(q=.25,axis = 1)
        # df['g_q50'] = df[features_g].quantile(q=.5,axis = 1)
        # df['g_q75'] = df[features_g].quantile(q=.75,axis = 1)
        #df['g_var'] = df[features_g].apply(axis=1,func=stats.variation)
        # df['g_mad'] = df[features_g].mad(axis = 1)


        df['c_sum'] = df[features_c].sum(axis = 1) ## <==
        # df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1) ## <==
        # df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        #df['c_skew'] = df[features_c].skew(axis = 1)
        # df['c_q25'] = df[features_c].quantile(q=.25,axis = 1)
        # df['c_q50'] = df[features_c].quantile(q=.5,axis = 1)
        # df['c_q75'] = df[features_c].quantile(q=.75,axis = 1)
        # df['c_var'] = df[features_c].apply(axis=1,func=stats.variation)
        # df['c_mad'] = df[features_c].mad(axis = 1)



        df['gc_sum'] = df[features_g + features_c].sum(axis = 1) ## <==
        # df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        # df['gc_std'] = df[features_g + features_c].std(axis = 1)
        # df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        # df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        # df['gc_q25'] = df[features_g + features_c].quantile(q=.25,axis = 1)
        # df['gc_q50'] = df[features_g + features_c].quantile(q=.5,axis = 1)
        # df['gc_q75'] = df[features_g + features_c].quantile(q=.75,axis = 1)
        # df['gc_var'] = df[features_g + features_c].apply(axis=1,func=stats.variation)
        # df['gc_mad'] = df[features_g + features_c].mad(axis = 1)


        
    return train, test,['c_sum']

train_features,test_features,STATS=fe_stats(train_features,test_features)

In [17]:
# train_features_gc = train_features[["sig_id"]+GENES+CELLS].copy()
# test_features_gc = test_features[["sig_id"]+GENES+CELLS].copy()

In [18]:
# from sklearn.feature_selection import VarianceThreshold


# var_thresh = VarianceThreshold(0.8)  #<-- Update
# data = train_features.append(test_features)
# data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

# train_features_transformed = data_transformed[ : train_features.shape[0]]
# test_features_transformed = data_transformed[-test_features.shape[0] : ]


# train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
#                               columns=['sig_id','cp_type','cp_time','cp_dose'])

# train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


# test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
#                              columns=['sig_id','cp_type','cp_time','cp_dose'])

# test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

# train_features.shape

In [19]:
# from sklearn.cluster import KMeans
# def fe_cluster(train, test, n_clusters_g = 15, n_clusters_c = 5, SEED = 42):
    
#     features_g = GENES
#     features_c = CELLS
    
#     def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
#         train_ = train[features].copy()
#         test_ = test[features].copy()
#         data = pd.concat([train_, test_], axis = 0)
#         kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
#         train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
#         test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
#         train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#         test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#         return train, test
    
#     #train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
#     train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
#     return train, test

# train_features_gc ,test_features_gc=fe_cluster(train_features_gc,test_features_gc)

In [20]:
from sklearn.cluster import KMeans
def fe_cluster2(train, test, n_clusters = 3, SEED = 42):
    

    def create_cluster(train, test, n_clusters = n_clusters):
        train_ = train.copy()
        test_ = test.copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data[[c for c in data.columns if c not in ["sig_id","cp_type","cp_dose","cp_time"]]])
        train['cluster'] = kmeans.labels_[:train.shape[0]]
        test['cluster'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = ['cluster'])
        test = pd.get_dummies(test, columns = ['cluster'])
        return train, test
    
    train, test = create_cluster(train, test, n_clusters = n_clusters)
    return train, test



# train_features,test_features=fe_cluster2(train_features,test_features)



In [21]:
# data = pd.concat([train1, test1], axis = 0)

# distortion = []
# for k in range(1,10):
#     kmeans = KMeans(n_clusters = k, random_state = 42).fit(data)
#     distortion += [kmeans.inertia_]


In [22]:
# plt.plot(range(1,10),distortion)

In [23]:
# train_features = train_features.merge(train_features_gc.loc[:,[col for col in train_features_gc.columns if col not in GENES+CELLS]],on="sig_id")
# test_features = test_features.merge(test_features_gc.loc[:,[col for col in test_features_gc.columns if col not in GENES+CELLS]],on="sig_id")

In [24]:
train_features.shape

(23814, 881)

In [25]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target_scored = train[train_targets_scored.columns]
target_nscored = train[train_targets_nonscored.columns]

In [26]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [27]:
target_scored_cols = target_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nscored_cols = target_nscored.drop('sig_id', axis=1).columns.values.tolist()

In [28]:
import torch
import numpy as np
from scipy.sparse import csc_matrix
import time
from abc import abstractmethod
#from pytorch_tabnet import tab_network
from pytorch_tabnet.multiclass_utils import unique_labels
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from torch.nn.utils import clip_grad_norm_
from pytorch_tabnet.utils import (PredictDataset,
                                  create_explain_matrix)
from sklearn.base import BaseEstimator
from torch.utils.data import DataLoader
from copy import deepcopy
import io
import json
from pathlib import Path
import shutil
import zipfile

In [29]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

def log_loss_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
    return results.mean()

def log_loss_score(actual, predicted,  eps=1e-15):

        """
        :param predicted:   The predicted probabilities as floats between 0-1
        :param actual:      The binary labels. Either 0 or 1.
        :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
        :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
        """

        
        p1 = actual * np.log(predicted+eps)
        p0 = (1-actual) * np.log(1-predicted+eps)
        loss = p0 + p1

        return -loss.mean()

In [30]:
class TrainDataset(Dataset):
    """
    Format for numpy array
    Parameters
    ----------
    X : 2D array
        The input matrix
    y : 2D array
        The one-hot encoded target
    """

    def __init__(self, x, y1,y2=None):
        self.x = x
        self.y1 = y1
        self.y2 = y2

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x, y1,y2 = self.x[index], self.y1[index], self.y2[index]
        return x, y1,y2

class ValidDataset(Dataset):
    """
    Format for numpy array
    Parameters
    ----------
    X : 2D array
        The input matrix
    y : 2D array
        The one-hot encoded target
    """

    def __init__(self, x, y1):
        self.x = x
        self.y1 = y1

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x, y1 = self.x[index], self.y1[index]
        return x, y1

In [31]:
def create_dataloaders(
    X_train, y_scored_train,y_nscored_train, X_valid,y_valid, weights, batch_size, num_workers, drop_last, pin_memory=True
):
    """
    Create dataloaders with or wihtout subsampling depending on weights and balanced.
    Parameters
    ----------
    X_train : np.ndarray
        Training data
    y_train : np.array
        Mapped Training targets
    eval_set : list of tuple
        List of eval tuple set (X, y)
    weights : either 0, 1, dict or iterable
        if 0 (default) : no weights will be applied
        if 1 : classification only, will balanced class with inverse frequency
        if dict : keys are corresponding class values are sample weights
        if iterable : list or np array must be of length equal to nb elements
                      in the training set
    batch_size : int
        how many samples per batch to load
    num_workers : int
        how many subprocesses to use for data loading. 0 means that the data
        will be loaded in the main process
    drop_last : bool
        set to True to drop the last incomplete batch, if the dataset size is not
        divisible by the batch size. If False and the size of dataset is not
        divisible by the batch size, then the last batch will be smaller
    pin_memory : bool
        Whether to pin GPU memory during training
    Returns
    -------
    train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
        Training and validation dataloaders
    """

    if isinstance(weights, int):
        if weights == 0:
            need_shuffle = True
            sampler = None
        elif weights == 1:
            need_shuffle = False
            class_sample_count = np.array(
                [len(np.where(y_train == t)[0]) for t in np.unique(y_scored_train)]
            )

            weights = 1.0 / class_sample_count

            samples_weight = np.array([weights[t] for t in y_scored_train])

            samples_weight = torch.from_numpy(samples_weight)
            samples_weight = samples_weight.double()
            sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
        else:
            raise ValueError("Weights should be either 0, 1, dictionnary or list.")
    elif isinstance(weights, dict):
        # custom weights per class
        need_shuffle = False
        samples_weight = np.array([weights[t] for t in y_scored_train])
        sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
    else:
        # custom weights
        if len(weights) != len(y_scored_train):
            raise ValueError("Custom weights should match number of train samples.")
        need_shuffle = False
        samples_weight = np.array(weights)
        sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

    train_dataloader = DataLoader(
        TrainDataset(X_train.astype(np.float32), y_scored_train, y_nscored_train),
        batch_size=batch_size,
        sampler=sampler,
        shuffle=need_shuffle,
        num_workers=num_workers,
        drop_last=drop_last,
        pin_memory=pin_memory
    )

    # valid_dataloaders = []
    # for X, y in [(X_valid,y_valid)]:
    #     valid_dataloaders.append(
    #         DataLoader(
    #             ValidDataset(X.astype(np.float32), y),
    #             batch_size=batch_size,
    #             shuffle=False,
    #             num_workers=num_workers,
    #             pin_memory=pin_memory
    #         )
    #     )

    valid_dataloaders = DataLoader(
        ValidDataset(X_valid.astype(np.float32), y_valid),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )


    return train_dataloader, valid_dataloaders

In [32]:
import torch
from torch.nn import Linear, BatchNorm1d, ReLU, PReLU,LeakyReLU
import numpy as np
from pytorch_tabnet import sparsemax


def initialize_non_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(4*input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


def initialize_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


class GBN(torch.nn.Module):
    """
        Ghost Batch Normalization
        https://arxiv.org/abs/1705.08741
    """

    def __init__(self, input_dim, virtual_batch_size=128, momentum=0.01):
        super(GBN, self).__init__()

        self.input_dim = input_dim
        self.virtual_batch_size = virtual_batch_size
        self.bn = BatchNorm1d(self.input_dim, momentum=momentum)

    def forward(self, x):
        chunks = x.chunk(int(np.ceil(x.shape[0] / self.virtual_batch_size)), 0)
        res = [self.bn(x_) for x_ in chunks]

        return torch.cat(res, dim=0)


class TabNetNoEmbeddings(torch.nn.Module):
    def __init__(self, input_dim, output_dim1,output_dim2,
                 n_d=8, n_a=8,
                 n_steps=3, gamma=1.3,
                 n_independent=2, n_shared=2, epsilon=1e-15,
                 virtual_batch_size=128, momentum=0.02,
                 mask_type="sparsemax"):
        """
        Defines main part of the TabNet network without the embedding layers.
        Parameters
        ----------
        input_dim : int
            Number of features
        output_dim : int or list of int for multi task classification
            Dimension of network output
            examples : one for regression, 2 for binary classification etc...
        n_d : int
            Dimension of the prediction  layer (usually between 4 and 64)
        n_a : int
            Dimension of the attention  layer (usually between 4 and 64)
        n_steps : int
            Number of sucessive steps in the newtork (usually betwenn 3 and 10)
        gamma : float
            Float above 1, scaling factor for attention updates (usually betwenn 1.0 to 2.0)
        n_independent : int
            Number of independent GLU layer in each GLU block (default 2)
        n_shared : int
            Number of independent GLU layer in each GLU block (default 2)
        epsilon : float
            Avoid log(0), this should be kept very low
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in all batch norm
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(TabNetNoEmbeddings, self).__init__()
        self.input_dim = input_dim
        self.output_dim1 = output_dim1
        self.output_dim2 = output_dim2
        self.is_multi_task = isinstance(output_dim1, list)
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.virtual_batch_size = virtual_batch_size
        self.mask_type = mask_type
        self.initial_bn = BatchNorm1d(self.input_dim, momentum=0.01)

        if self.n_shared > 0:
            shared_feat_transform = torch.nn.ModuleList()
            for i in range(self.n_shared):
                if i == 0:
                    shared_feat_transform.append(Linear(self.input_dim,
                                                        2*(n_d + n_a),
                                                        bias=False))
                else:
                    shared_feat_transform.append(Linear(n_d + n_a, 2*(n_d + n_a), bias=False))

        else:
            shared_feat_transform = None

        self.initial_splitter = FeatTransformer(self.input_dim, n_d+n_a, shared_feat_transform,
                                                n_glu_independent=self.n_independent,
                                                virtual_batch_size=self.virtual_batch_size,
                                                momentum=momentum)

        self.feat_transformers = torch.nn.ModuleList()
        self.att_transformers = torch.nn.ModuleList()

        for step in range(n_steps):
            transformer = FeatTransformer(self.input_dim, n_d+n_a, shared_feat_transform,
                                          n_glu_independent=self.n_independent,
                                          virtual_batch_size=self.virtual_batch_size,
                                          momentum=momentum)
            attention = AttentiveTransformer(n_a, self.input_dim,
                                             virtual_batch_size=self.virtual_batch_size,
                                             momentum=momentum,
                                             mask_type=self.mask_type)
            self.feat_transformers.append(transformer)
            self.att_transformers.append(attention)

        if self.is_multi_task:
            self.multi_task_mappings1 = torch.nn.ModuleList()
            for task_dim in output_dim1:
                task_mapping = Linear(n_d, task_dim, bias=False)
                initialize_non_glu(task_mapping, n_d, task_dim)
                self.multi_task_mappings1.append(task_mapping)

            # self.multi_task_mappings2 = torch.nn.ModuleList()
            # for task_dim in output_dim2:
            #     task_mapping = Linear(n_d, task_dim, bias=False)
            #     initialize_non_glu(task_mapping, n_d, task_dim)
            #     self.multi_task_mappings2.append(task_mapping)
        else:
            self.final_mapping1 = Linear(n_d, output_dim1, bias=False)
            initialize_non_glu(self.final_mapping1, n_d, output_dim1)
            # self.final_mapping2 = Linear(n_d, output_dim2, bias=False)
            # initialize_non_glu(self.final_mapping2, n_d, output_dim2)

    def forward(self, x):
        res = 0
        x = self.initial_bn(x)

        prior = torch.ones(x.shape).to(x.device)
        M_loss = 0
        att = self.initial_splitter(x)[:, self.n_d:]

        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)
            M_loss += torch.mean(torch.sum(torch.mul(M, torch.log(M+self.epsilon)),
                                           dim=1))
            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            masked_x = torch.mul(M, x)
            out = self.feat_transformers[step](masked_x)
            d = ReLU()(out[:, :self.n_d])
            res = torch.add(res, d)
            # update attention
            att = out[:, self.n_d:]

        M_loss /= self.n_steps

        if self.is_multi_task:
            # Result will be in list format
            out1 = []
            for task_mapping in self.multi_task_mappings1:
                out1.append(task_mapping(res))
            # out2 = []
            # for task_mapping in self.multi_task_mappings2:
            #     out2.append(task_mapping(res))
        else:
            out1 = self.final_mapping1(res)
            # out2 = self.final_mapping2(res)
        # return out1,out2, M_loss
        return out1, M_loss

    def forward_masks(self, x):
        x = self.initial_bn(x)

        prior = torch.ones(x.shape).to(x.device)
        M_explain = torch.zeros(x.shape).to(x.device)
        att = self.initial_splitter(x)[:, self.n_d:]
        masks = {}

        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)
            masks[step] = M
            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            masked_x = torch.mul(M, x)
            out = self.feat_transformers[step](masked_x)
            d = ReLU()(out[:, :self.n_d])
            # explain
            step_importance = torch.sum(d, dim=1)
            M_explain += torch.mul(M, step_importance.unsqueeze(dim=1))
            # update attention
            att = out[:, self.n_d:]

        return M_explain, masks


class TabNet(torch.nn.Module):
    def __init__(self, input_dim, output_dim1,output_dim2, n_d=8, n_a=8,
                 n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,
                 virtual_batch_size=128, momentum=0.02, device_name='auto',
                 mask_type="sparsemax"):
        """
        Defines TabNet network
        Parameters
        ----------
        input_dim : int
            Initial number of features
        output_dim : int
            Dimension of network output
            examples : one for regression, 2 for binary classification etc...
        n_d : int
            Dimension of the prediction  layer (usually between 4 and 64)
        n_a : int
            Dimension of the attention  layer (usually between 4 and 64)
        n_steps : int
            Number of sucessive steps in the newtork (usually betwenn 3 and 10)
        gamma : float
            Float above 1, scaling factor for attention updates (usually betwenn 1.0 to 2.0)
        cat_idxs : list of int
            Index of each categorical column in the dataset
        cat_dims : list of int
            Number of categories in each categorical column
        cat_emb_dim : int or list of int
            Size of the embedding of categorical features
            if int, all categorical features will have same embedding size
            if list of int, every corresponding feature will have specific size
        n_independent : int
            Number of independent GLU layer in each GLU block (default 2)
        n_shared : int
            Number of independent GLU layer in each GLU block (default 2)
        epsilon : float
            Avoid log(0), this should be kept very low
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in all batch norm
        device_name : {'auto', 'cuda', 'cpu'}
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(TabNet, self).__init__()
        self.cat_idxs = cat_idxs or []
        self.cat_dims = cat_dims or []
        self.cat_emb_dim = cat_emb_dim

        self.input_dim = input_dim
        self.output_dim1 = output_dim1
        self.output_dim2 = output_dim2
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.mask_type = mask_type

        if self.n_steps <= 0:
            raise ValueError("n_steps should be a positive integer.")
        if self.n_independent == 0 and self.n_shared == 0:
            raise ValueError("n_shared and n_independant can't be both zero.")

        self.virtual_batch_size = virtual_batch_size
        self.embedder = EmbeddingGenerator(input_dim, cat_dims, cat_idxs, cat_emb_dim)
        self.post_embed_dim = self.embedder.post_embed_dim
        self.tabnet = TabNetNoEmbeddings(self.post_embed_dim, output_dim1,output_dim2, n_d, n_a, n_steps,
                                         gamma, n_independent, n_shared, epsilon,
                                         virtual_batch_size, momentum, mask_type)

        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        self.to(self.device)

    def forward(self, x):
        x = self.embedder(x)
        return self.tabnet(x)

    def forward_masks(self, x):
        x = self.embedder(x)
        return self.tabnet.forward_masks(x)


class AttentiveTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim,
                 virtual_batch_size=128,
                 momentum=0.02,
                 mask_type="sparsemax"):
        """
        Initialize an attention transformer.
        Parameters
        ----------
        input_dim : int
            Input size
        output_dim : int
            Outpu_size
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(AttentiveTransformer, self).__init__()
        self.fc = Linear(input_dim, output_dim, bias=False)
        initialize_non_glu(self.fc, input_dim, output_dim)
        self.bn = GBN(output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)

        if mask_type == "sparsemax":
            # Sparsemax
            self.selector = sparsemax.Sparsemax(dim=-1)
        elif mask_type == "entmax":
            # Entmax
            self.selector = sparsemax.Entmax15(dim=-1)
        else:
            raise NotImplementedError("Please choose either sparsemax" +
                                      "or entmax as masktype")

    def forward(self, priors, processed_feat):
        x = self.fc(processed_feat)
        x = self.bn(x)
        x = torch.mul(x, priors)
        x = self.selector(x)
        return x


class FeatTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, shared_layers, n_glu_independent,
                 virtual_batch_size=128, momentum=0.02):
        super(FeatTransformer, self).__init__()
        """
        Initialize a feature transformer.
        Parameters
        ----------
        input_dim : int
            Input size
        output_dim : int
            Outpu_size
        shared_layers : torch.nn.ModuleList
            The shared block that should be common to every step
        n_glu_independant : int
            Number of independent GLU layers
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization within GLU block(s)
        momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        """

        params = {
            'n_glu': n_glu_independent,
            'virtual_batch_size': virtual_batch_size,
            'momentum': momentum
        }

        if shared_layers is None:
            # no shared layers
            self.shared = torch.nn.Identity()
            is_first = True
        else:
            self.shared = GLU_Block(input_dim, output_dim,
                                    first=True,
                                    shared_layers=shared_layers,
                                    n_glu=len(shared_layers),
                                    virtual_batch_size=virtual_batch_size,
                                    momentum=momentum)
            is_first = False

        if n_glu_independent == 0:
            # no independent layers
            self.specifics = torch.nn.Identity()
        else:
            spec_input_dim = input_dim if is_first else output_dim
            self.specifics = GLU_Block(spec_input_dim, output_dim,
                                       first=is_first,
                                       **params)

    def forward(self, x):
        x = self.shared(x)
        x = self.specifics(x)
        return x


class GLU_Block(torch.nn.Module):
    """
        Independant GLU block, specific to each step
    """

    def __init__(self, input_dim, output_dim, n_glu=2, first=False, shared_layers=None,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Block, self).__init__()
        self.first = first
        self.shared_layers = shared_layers
        self.n_glu = n_glu
        self.glu_layers = torch.nn.ModuleList()

        params = {
            'virtual_batch_size': virtual_batch_size,
            'momentum': momentum
        }

        fc = shared_layers[0] if shared_layers else None
        self.glu_layers.append(GLU_Layer(input_dim, output_dim,
                                         fc=fc,
                                         **params))
        for glu_id in range(1, self.n_glu):
            fc = shared_layers[glu_id] if shared_layers else None
            self.glu_layers.append(GLU_Layer(output_dim, output_dim,
                                             fc=fc,
                                             **params))

    def forward(self, x):
        scale = torch.sqrt(torch.FloatTensor([0.5]).to(x.device))
        if self.first:  # the first layer of the block has no scale multiplication
            x = self.glu_layers[0](x)
            layers_left = range(1, self.n_glu)
        else:
            layers_left = range(self.n_glu)

        for glu_id in layers_left:
            x = torch.add(x, self.glu_layers[glu_id](x))
            x = x*scale
        return x


class GLU_Layer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, fc=None,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Layer, self).__init__()

        self.output_dim = output_dim
        if fc:
            self.fc = fc
        else:
            self.fc = Linear(input_dim, 2*output_dim, bias=False)
        initialize_glu(self.fc, input_dim, 2*output_dim)

        self.bn = GBN(2*output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        out = torch.mul(x[:, :self.output_dim], torch.sigmoid(x[:, self.output_dim:]))
        return out


class EmbeddingGenerator(torch.nn.Module):
    """
        Classical embeddings generator
    """

    def __init__(self, input_dim, cat_dims, cat_idxs, cat_emb_dim):
        """ This is an embedding module for an entier set of features
        Parameters
        ----------
        input_dim : int
            Number of features coming as input (number of columns)
        cat_dims : list of int
            Number of modalities for each categorial features
            If the list is empty, no embeddings will be done
        cat_idxs : list of int
            Positional index for each categorical features in inputs
        cat_emb_dim : int or list of int
            Embedding dimension for each categorical features
            If int, the same embdeding dimension will be used for all categorical features
        """
        super(EmbeddingGenerator, self).__init__()
        if cat_dims == [] or cat_idxs == []:
            self.skip_embedding = True
            self.post_embed_dim = input_dim
            return

        self.skip_embedding = False
        if isinstance(cat_emb_dim, int):
            self.cat_emb_dims = [cat_emb_dim]*len(cat_idxs)
        else:
            self.cat_emb_dims = cat_emb_dim

        # check that all embeddings are provided
        if len(self.cat_emb_dims) != len(cat_dims):
            msg = """ cat_emb_dim and cat_dims must be lists of same length, got {len(self.cat_emb_dims)}
                      and {len(cat_dims)}"""
            raise ValueError(msg)
        self.post_embed_dim = int(input_dim + np.sum(self.cat_emb_dims) - len(self.cat_emb_dims))

        self.embeddings = torch.nn.ModuleList()

        # Sort dims by cat_idx
        sorted_idxs = np.argsort(cat_idxs)
        cat_dims = [cat_dims[i] for i in sorted_idxs]
        self.cat_emb_dims = [self.cat_emb_dims[i] for i in sorted_idxs]

        for cat_dim, emb_dim in zip(cat_dims, self.cat_emb_dims):
            self.embeddings.append(torch.nn.Embedding(cat_dim, emb_dim))

        # record continuous indices
        self.continuous_idx = torch.ones(input_dim, dtype=torch.bool)
        self.continuous_idx[cat_idxs] = 0

    def forward(self, x):
        """
        Apply embdeddings to inputs
        Inputs should be (batch_size, input_dim)
        Outputs will be of size (batch_size, self.post_embed_dim)
        """
        if self.skip_embedding:
            # no embeddings required
            return x

        cols = []
        cat_feat_counter = 0
        for feat_init_idx, is_continuous in enumerate(self.continuous_idx):
            # Enumerate through continuous idx boolean mask to apply embeddings
            if is_continuous:
                cols.append(x[:, feat_init_idx].float().view(-1, 1))
            else:
                cols.append(self.embeddings[cat_feat_counter](x[:, feat_init_idx].long()))
                cat_feat_counter += 1
        # concat
        post_embeddings = torch.cat(cols, dim=1)
        return post_embeddings

In [33]:
class TabModel(BaseEstimator):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,  momentum=0.02,
                 lambda_sparse=1e-3, seed=0,
                 clip_value=1, verbose=1,
                 optimizer_fn=torch.optim.Adam,
                 optimizer_params=dict(lr=2e-2),
                 scheduler_params=None, scheduler_fn=None,
                 mask_type="sparsemax",
                 input_dim=None, output_dim1=None,output_dim2=None,
                 device_name='auto'):
        """ Class for TabNet model
        Parameters
        ----------
            device_name: str
                'cuda' if running on GPU, 'cpu' if not, 'auto' to autodetect
        """

        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.cat_idxs = cat_idxs
        self.cat_dims = cat_dims
        self.cat_emb_dim = cat_emb_dim
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.epsilon = epsilon
        self.momentum = momentum
        self.lambda_sparse = lambda_sparse
        self.clip_value = clip_value
        self.verbose = verbose
        self.optimizer_fn = optimizer_fn
        self.optimizer_params = optimizer_params
        self.device_name = device_name
        self.scheduler_params = scheduler_params
        self.scheduler_fn = scheduler_fn
        self.mask_type = mask_type
        self.input_dim = input_dim
        self.output_dim1 = output_dim1
        self.output_dim2 = output_dim2

        #self.batch_size = 1024
        self.batch_size = 2048

        self.seed = seed
        torch.manual_seed(self.seed)
        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        print(f"Device used : {self.device}")

    @abstractmethod
    def construct_loaders(self, X_train, y_scored_train,y_nscored_train, X_valid, y_valid,
                          weights, batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        raise NotImplementedError('users must define construct_loaders to use this base class')

    def init_network(
                     self,
                     input_dim,
                     output_dim1,
                     output_dim2,
                     n_d,
                     n_a,
                     n_steps,
                     gamma,
                     cat_idxs,
                     cat_dims,
                     cat_emb_dim,
                     n_independent,
                     n_shared,
                     epsilon,
                     virtual_batch_size,
                     momentum,
                     device_name,
                     mask_type,
                     ):
        self.network = TabNet(
            input_dim,
            output_dim1,
            output_dim2,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            device_name=device_name,
            mask_type=mask_type).to(self.device)

        self.reducing_matrix = create_explain_matrix(
            self.network.input_dim,
            self.network.cat_emb_dim,
            self.network.cat_idxs,
            self.network.post_embed_dim)

    def fit(self, X_train, y_scored_train, y_nscored_train, X_valid=None, y_valid=None, loss_fn=None,loss_tr=None,
            weights=0, max_epochs=100, patience=10, batch_size=1024,
            virtual_batch_size=128, num_workers=0, drop_last=False):
        """Train a neural network stored in self.network
        Using train_dataloader for training data and
        valid_dataloader for validation.
        Parameters
        ----------
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            weights : bool or dictionnary
                0 for no balancing
                1 for automated balancing
                dict for custom weights per class
            max_epochs : int
                Maximum number of epochs during training
            patience : int
                Number of consecutive non improving epoch before early stopping
            batch_size : int
                Training batch size
            virtual_batch_size : int
                Batch size for Ghost Batch Normalization (virtual_batch_size < batch_size)
            num_workers : int
                Number of workers used in torch.utils.data.DataLoader
            drop_last : bool
                Whether to drop last batch during training
        """
        # update model name

        self.update_fit_params(X_train, y_scored_train,y_nscored_train, X_valid, y_valid, loss_fn,loss_tr,
                               weights, max_epochs, patience, batch_size,virtual_batch_size, num_workers, drop_last)


        train_dataloader, valid_dataloader = self.construct_loaders(X_train,
                                                                    y_scored_train,
                                                                    y_nscored_train,
                                                                    X_valid,
                                                                    y_valid,
                                                                    self.updated_weights,
                                                                    self.batch_size,
                                                                    self.num_workers,
                                                                    self.drop_last)

        self.init_network(
            input_dim=self.input_dim,
            output_dim1=self.output_dim1,
            output_dim2=self.output_dim2,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=self.virtual_batch_size,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )

        self.optimizer = self.optimizer_fn(self.network.parameters(),
                                           **self.optimizer_params)

        if self.scheduler_fn:
            self.scheduler = self.scheduler_fn(self.optimizer, **self.scheduler_params)
        else:
            self.scheduler = None

        self.losses_train = []
        self.losses_valid = []
        self.learning_rates = []
        self.metrics_train = []
        self.metrics_valid = []

        if self.verbose > 0:
            print("Will train until validation stopping metric",
                  f"hasn't improved in {self.patience} rounds.")
            msg_epoch = f'| EPOCH |  train  |   valid  | total time (s)'
            print('---------------------------------------')
            print(msg_epoch)

        total_time = 0
        while (self.epoch < self.max_epochs and
               self.patience_counter < self.patience):
            starting_time = time.time()
            # updates learning rate history
            self.learning_rates.append(self.optimizer.param_groups[-1]["lr"])

            fit_metrics = self.fit_epoch(train_dataloader, valid_dataloader)

            # leaving it here, may be used for callbacks later
            self.losses_train.append(fit_metrics['train']['loss_avg'])
            self.losses_valid.append(fit_metrics['valid']['total_loss'])
            self.metrics_train.append(fit_metrics['train']['stopping_loss'])
            self.metrics_valid.append(fit_metrics['valid']['stopping_loss'])

            stopping_loss = fit_metrics['valid']['stopping_loss']
            if stopping_loss < self.best_cost:
                self.best_cost = stopping_loss
                self.patience_counter = 0
                # Saving model
                self.best_network = deepcopy(self.network)
                has_improved = True
            else:
                self.patience_counter += 1
                has_improved=False
            self.epoch += 1
            total_time += time.time() - starting_time
            if self.verbose > 0:
                if self.epoch % self.verbose == 0:
                    separator = "|"
                    msg_epoch = f"| {self.epoch:<5} | "
                    msg_epoch += f" {fit_metrics['train']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {fit_metrics['valid']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {np.round(total_time, 1):<10}"
                    msg_epoch += f" {has_improved}"
                    print(msg_epoch)

        if self.verbose > 0:
            if self.patience_counter == self.patience:
                print(f"Early stopping occured at epoch {self.epoch}")
            print(f"Training done in {total_time:.3f} seconds.")
            print('---------------------------------------')

        self.history = {"train": {"loss": self.losses_train,
                                  "metric": self.metrics_train,
                                  "lr": self.learning_rates},
                        "valid": {"loss": self.losses_valid,
                                  "metric": self.metrics_valid}}
        # load best models post training
        self.load_best_model()

        # compute feature importance once the best model is defined
        self._compute_feature_importances(train_dataloader)

    def save_model(self, path):
        """
        Saving model with two distinct files.
        """
        saved_params = {}
        for key, val in self.get_params().items():
            if isinstance(val, type):
                # Don't save torch specific params
                continue
            else:
                saved_params[key] = val

        # Create folder
        Path(path).mkdir(parents=True, exist_ok=True)

        # Save models params
        with open(Path(path).joinpath("model_params.json"), "w", encoding="utf8") as f:
            json.dump(saved_params, f)

        # Save state_dict
        torch.save(self.network.state_dict(), Path(path).joinpath("network.pt"))
        shutil.make_archive(path, 'zip', path)
        shutil.rmtree(path)
        print(f"Successfully saved model at {path}.zip")
        return f"{path}.zip"

    def load_model(self, filepath):

        try:
            try:
                with zipfile.ZipFile(filepath) as z:
                    with z.open("model_params.json") as f:
                        loaded_params = json.load(f)
                    with z.open("network.pt") as f:
                        try:
                            saved_state_dict = torch.load(f)
                        except io.UnsupportedOperation:
                            # In Python <3.7, the returned file object is not seekable (which at least
                            # some versions of PyTorch require) - so we'll try buffering it in to a
                            # BytesIO instead:
                            saved_state_dict = torch.load(io.BytesIO(f.read()))
                            
            except:
                with open(os.path.join(filepath, "model_params.json")) as f:
                        loaded_params = json.load(f)

                saved_state_dict = torch.load(os.path.join(filepath, "network.pt"), map_location="cpu")
 
        except KeyError:
            raise KeyError("Your zip file is missing at least one component")

        #print(loaded_params)
        if torch.cuda.is_available():
            device_name = 'cuda'
        else:
            device_name = 'cpu'
        loaded_params["device_name"] = device_name
        self.__init__(**loaded_params)
        
        

        self.init_network(
            input_dim=self.input_dim,
            output_dim1=self.output_dim1,
            output_dim2=self.output_dim2,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=1024,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )
        self.network.load_state_dict(saved_state_dict)
        self.network.eval()
        return

    def fit_epoch(self, train_dataloader, valid_dataloader):
        """
        Evaluates and updates network for one epoch.
        Parameters
        ----------
            train_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
            valid_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with valid set
        """
        train_metrics = self.train_epoch(train_dataloader)
        valid_metrics = self.predict_epoch(valid_dataloader)

        fit_metrics = {'train': train_metrics,
                       'valid': valid_metrics}

        return fit_metrics

    @abstractmethod
    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """
        raise NotImplementedError('users must define train_epoch to use this base class')

    @abstractmethod
    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        raise NotImplementedError('users must define train_batch to use this base class')

    @abstractmethod
    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        raise NotImplementedError('users must define predict_epoch to use this base class')

    @abstractmethod
    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        raise NotImplementedError('users must define predict_batch to use this base class')

    def load_best_model(self):
        if self.best_network is not None:
            self.network = self.best_network

    @abstractmethod
    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem or the last class
        """
        raise NotImplementedError('users must define predict to use this base class')

    def explain(self, X):
        """
        Return local explanation
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            M_explain: matrix
                Importance per sample, per columns.
            masks: matrix
                Sparse matrix showing attention masks used by network.
        """
        self.network.eval()

        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            M_explain, masks = self.network.forward_masks(data)
            for key, value in masks.items():
                masks[key] = csc_matrix.dot(value.cpu().detach().numpy(),
                                            self.reducing_matrix)

            if batch_nb == 0:
                res_explain = csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                             self.reducing_matrix)
                res_masks = masks
            else:
                res_explain = np.vstack([res_explain,
                                         csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                                        self.reducing_matrix)])
                for key, value in masks.items():
                    res_masks[key] = np.vstack([res_masks[key], value])
        return res_explain, res_masks

    def _compute_feature_importances(self, loader):
        self.network.eval()
        feature_importances_ = np.zeros((self.network.post_embed_dim))
        for data, targets,_ in loader:
            data = data.to(self.device).float()
            M_explain, masks = self.network.forward_masks(data)
            feature_importances_ += M_explain.sum(dim=0).cpu().detach().numpy()

        feature_importances_ = csc_matrix.dot(feature_importances_,
                                              self.reducing_matrix)
        self.feature_importances_ = feature_importances_ / np.sum(feature_importances_)
        


class TabNetRegressor(TabModel):

    def construct_loaders(self, X_train, y_scored_train,y_nscored_train, X_valid, y_valid, weights,
                          batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        if isinstance(weights, int):
            if weights == 1:
                raise ValueError("Please provide a list of weights for regression.")
        if isinstance(weights, dict):
            raise ValueError("Please provide a list of weights for regression.")



        train_dataloader, valid_dataloader = create_dataloaders(X_train,
                                                                y_scored_train,
                                                                y_nscored_train,
                                                                X_valid,
                                                                y_valid,
                                                                weights,
                                                                batch_size,
                                                                num_workers,
                                                                drop_last)
        return train_dataloader, valid_dataloader

    def update_fit_params(self, X_train, y_scored_train, y_nscored_train, X_valid, y_valid, loss_fn, loss_tr,
                          weights, max_epochs, patience,batch_size, virtual_batch_size, num_workers, drop_last):


        if loss_fn is None:
            self.loss_fn = torch.nn.functional.mse_loss
        else:
            self.loss_fn = loss_fn
            self.loss_tr = loss_tr

        assert X_train.shape[1] == X_valid.shape[1], "Dimension mismatch X_train X_valid"
        self.input_dim = X_train.shape[1]

        if len(y_scored_train.shape) == 1:
            raise ValueError("""Please apply reshape(-1, 1) to your targets
                                if doing single regression.""")
        assert y_scored_train.shape[1] == y_valid.shape[1], "Dimension mismatch y_train y_valid"
        self.output_dim1 = y_scored_train.shape[1]
        self.output_dim2 = y_nscored_train.shape[1]

        self.updated_weights = weights

        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        # Initialize counters and histories.
        self.patience_counter = 0
        self.epoch = 0
        self.best_cost = np.inf
        self.num_workers = num_workers
        self.drop_last = drop_last

    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """

        self.network.train()
        y_preds = []
        ys = []
        total_loss = 0

        for data, targets_scored, targets_nscored in train_loader:
            batch_outs = self.train_batch(data, targets_scored, targets_nscored)
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())
            total_loss += batch_outs["loss"]

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        #stopping_loss = mean_squared_error(y_true=ys, y_pred=y_preds)
        # stopping_loss =log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  )
        total_loss = total_loss / len(train_loader)

        epoch_metrics = {'loss_avg': total_loss,
                         'stopping_loss': total_loss,
                         }

        if self.scheduler is not None:
            self.scheduler.step()
            
        return epoch_metrics

    def train_batch(self, data, targets_scored, targets_nscored):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        self.network.train()
        data = data.to(self.device).float()

        

        targets_scored = targets_scored.to(self.device).float()
        targets_nscored = targets_nscored.to(self.device).float()

        self.optimizer.zero_grad()

        # output1,output2, M_loss = self.network(data)
        output1, M_loss = self.network(data)

        loss1 = self.loss_fn(output1, targets_scored)
        # loss2 = self.loss_fn(output2, targets_nscored)
        loss = loss1 #+ loss2
        
        loss -= self.lambda_sparse*M_loss

        loss.backward()
        if self.clip_value:
            clip_grad_norm_(self.network.parameters(), self.clip_value)
        self.optimizer.step()

        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output1,
                      'y': targets_scored}
        return batch_outs

    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        y_preds = []
        ys = []
        self.network.eval()
        total_loss = 0

        for data, targets in loader:
            batch_outs = self.predict_batch(data, targets)
            total_loss += batch_outs["loss"]
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        stopping_loss = log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  ) #mean_squared_error(y_true=ys, y_pred=y_preds)

        # if self.scheduler is not None:
        #     self.scheduler.step(stopping_loss)

        total_loss = total_loss / len(loader)
        epoch_metrics = {'total_loss': total_loss,
                         'stopping_loss': stopping_loss}

        return epoch_metrics

    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        self.network.eval()
        data = data.to(self.device).float()
        targets = targets.to(self.device).float()

        #output,_, M_loss = self.network(data)
        output,M_loss = self.network(data)
       
        loss = self.loss_fn(output, targets)
        #print(self.loss_fn, loss)
        loss -= self.lambda_sparse*M_loss
        #print(loss)
        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem
        """
        self.network.eval()
        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        results = []
        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            #output,_, M_loss = self.network(data)
            output, M_loss = self.network(data)
            predictions = output.cpu().detach().numpy()
            results.append(predictions)
        res = np.vstack(results)
        return res

In [34]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"

EPOCHS = 300

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5

BATCH_SIZE = 4096

WEIGHTS=(1/np.mean(train_targets_scored[[c for c in  train_targets_scored.columns if c != "sig_id"]],axis=0)).values

cat_cols = ['cluster_0',"cluster_1",'cluster_2','cp_time_24', 'cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2']

# Parameters model
cat_emb_dim=[1] * len(cat_cols) #to choose
cats_idx = [1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046]
cat_dims = [2] * len(cat_cols) #to choose



save_name = "../data/tabnet-weights-public/tabnet-raw-public-step1/tabnet_raw_step1"

In [36]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=NFOLDS)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target_scored)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

,sig_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor,kfold
0,id_000644bb2,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,0,0,0,0,0,0,0,0,0,1
3,id_0015fd391,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,0,0,0,0,0,0,0,0,0,2
4,id_001626bd3,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,-0.1166,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0,0,0,0,0,0,0,0,0,4
21945,id_fffb70c0c,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,...,0,0,0,0,0,0,0,0,0,1


In [74]:
#feature_cols = [c for c in process_data(folds).columns if c not in (target_scored_cols + target_nscored_cols)]
#feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]

one_hot_col = ['cp_time_24','cp_time_48', 'cp_time_72', 'cp_dose_D1', 'cp_dose_D2']

feature_cols = [c for c in process_data(folds).columns if c in (GENES + CELLS)]
len(feature_cols)

872

In [75]:
# Parameters model
cat_emb_dim=[1] * len(cat_cols) #to choose
cats_idx = [872,873,874,875,876]
cat_dims = [2] * len(cat_cols) #to choose

In [76]:
import math

def run_training(fold, seed):

    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    

    x_train, y_scored_train, y_nscored_train  = train_df[feature_cols].values, train_df[target_scored_cols].values, train_df[target_nscored_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_scored_cols].values
    

    

    model = TabNetRegressor(n_d=24, 
                            n_a=24, 
                            n_steps=1, 
                            gamma=1.3,
                            # cat_dims=cat_dims, 
                            # cat_emb_dim=cat_emb_dim, 
                            # cat_idxs=cats_idx,
                            lambda_sparse=0, 
                            optimizer_fn=torch.optim.Adam,
                            optimizer_params=dict(lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY), 
                            mask_type='entmax', 
                            device_name=device, 
                            scheduler_params=dict(pct_start=0.1, div_factor=1e4, final_div_factor=1e5,
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=math.ceil(x_train.shape[0]/BATCH_SIZE)), 
                            scheduler_fn=torch.optim.lr_scheduler.OneCycleLR)


    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)

    model.fit(X_train=x_train, 
              y_scored_train=y_scored_train,
              y_nscored_train=y_nscored_train,  
              X_valid=x_valid, 
              y_valid=y_valid,
              max_epochs=EPOCHS,
              patience=EPOCHS, 
              batch_size=BATCH_SIZE, 
              virtual_batch_size=128,
              num_workers=0, 
              drop_last=False,
              weights=0,
              loss_fn=loss_fn,
              loss_tr=loss_tr)


    oof = np.zeros((len(train), target_scored.iloc[:, 1:].shape[1]))
    
    model.load_best_model()
    preds = model.predict(x_valid)
    oof[val_idx] = torch.sigmoid(torch.as_tensor(preds)).detach().cpu().numpy()

    x_test = test_[feature_cols].values
    preds = model.predict(x_test)
    predictions = torch.sigmoid(torch.as_tensor(preds)).detach().cpu().numpy()
    
    return oof, predictions

In [77]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_scored_cols)))
    predictions = np.zeros((len(test), len(target_scored_cols)))
    
    for fold in range(NFOLDS):
        print(f"SEED {SEED} - FOLD {fold}")
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions


In [78]:
# Averaging on multiple SEEDS

# SEED = [0,1,2,3,4,5,6] #<-- Update
SEED = [0]
oof = np.zeros((len(train), len(target_scored_cols)))
predictions = np.zeros((len(test), len(target_scored_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_scored_cols] = oof
test[target_scored_cols] = predictions

alse
| 220   |  0.01657 |   0.01894 |   331.3      False
| 221   |  0.01654 |   0.01886 |   332.6      False
| 222   |  0.01649 |   0.01899 |   334.0      False
| 223   |  0.01637 |   0.01888 |   335.2      False
| 224   |  0.01623 |   0.01897 |   336.5      False
| 225   |  0.01631 |   0.01881 |   337.7      False
| 226   |  0.01624 |   0.01895 |   339.1      False
| 227   |  0.01619 |   0.01928 |   340.3      False
| 228   |  0.01626 |   0.01898 |   341.8      False
| 229   |  0.01633 |   0.01896 |   343.1      False
| 230   |  0.01606 |   0.01900 |   344.6      False
| 231   |  0.01617 |   0.01958 |   346.1      False
| 232   |  0.01589 |   0.01909 |   347.2      False
| 233   |  0.01590 |   0.01902 |   348.5      False
| 234   |  0.01577 |   0.01908 |   349.7      False
| 235   |  0.01571 |   0.01906 |   351.9      False
| 236   |  0.01574 |   0.01920 |   353.6      False
| 237   |  0.01577 |   0.01928 |   354.8      False
| 238   |  0.01584 |   0.01924 |   356.4      False
| 239  

In [81]:
valid_results = train_targets_scored.drop(columns=target_scored_cols).merge(train[['sig_id']+target_scored_cols], on='sig_id', how='left').fillna(0)



y_true = train_targets_scored[target_scored_cols].values
y_pred = valid_results[target_scored_cols].values

score = 0
for i in range(len(target_scored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target_scored.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.017049978439132536


In [80]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

0.6370478913250259


In [43]:

#baseline model CV log_loss:  0.017229214153524654 0.636904475615476
#without loss2 - CV log_loss:  0.017113850278195217 0.6411332576440505
#with loss1+loss2 n_d=32, n_d=32 - CV log_loss:  0.017371728589871866 0.6307375162361144
#without loss2 -batchsize 4096 - CV log_loss:  0.018033571525737865 - 0.5979066038623788 
#without loss2 -batchsize 512  - CV log_loss:  0.018299183988458336 - 0.5957070411145741
#with loss1+loss2 - batchsize 4096 - CV log_loss:  0.017333979807842674 - 0.6355101856336954
#with loss1+loss2 - batchsize 4096 - no-embedding CV log_loss:  0.017958707605862843 - 0.6090030784964461
#with loss1+loss2 - batchsize 4096 - no-embedding and no cat features - CV log_loss:  0.017790546282443625 - 0.6122124872950352
#with loss1+loss2 - batchsize 4096 - no-embedding and no cat features - no custom featurees - CV log_loss:  0.01742562179948169 - 0.6287034941247442
#without anything but raw features (loss1 + loss2) : CV log_loss:  0.01671694783828676 - 0.6677187540881376
#without loss2 - CV log_loss:  0.016489743520331716 - 0.6801202711309785
#without loss2 - 400 epochs : CV log_loss:  0.01674019648108219 - 0.665161484053437
#without loss2 - batch 2048 - CV log_loss:  0.016965090255774976 - 0.6543266467935491
#without loss2 - batch 7192 - CCV log_loss:  0.01730564431591043 0.6378402328849997
# virtual_batch_size=256 CV log_loss:  0.017089849123627335 0.6439813945789904
# leaky relu 1 - CV log_loss:  0.016935782616855233 0.6498754729482525
# leaky relu 2 - CV log_loss:  0.017049978439132536 0.6370478913250259
# nd na = 20   - CV log_loss:  0.01703242968172429  0.6417327687598686